# Optimization of JKR NN regression model with Optuna

In [562]:
import optuna
from optuna.trial import TrialState

# Math and Dataframes
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import PercentFormatter

# Machine Learning 
from sklearn.model_selection import train_test_split
import torch
import torch.optim as optim
from torch import nn
# from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

# Others
from datetime import datetime
import os
from pathlib import Path

In [563]:
def hertz(i, E, nu, r):
    a = i/r
    factor = 1 - 0.1 * a - (1/840) * a**2 + (11/15120) * a**3 + (1357/6652800) * a**4
    force = 4/3 * E / (1 - nu**2) * np.sqrt(r)*i**1.5 * factor
    # make nan values zero
    force[np.isnan(force)] = 0
    return force*10**-6

def jkr(i, E, nu, gamma, r):
    E = E * 10**3 # kPa to Pa
    i = i * 10**-9 # nm to m
    r = r * 10**-9 # nm to m
    gamma = gamma * 10**-6 # microJ/m^2 to J/m^2
    E_eff = E / (1 - nu**2)
    K = 4/3 * E_eff
    Ua = np.sqrt(6*np.pi*gamma)
    # JKR force formula in (3)
    force = K * r **0.5 * i**1.5 - Ua * K**0.5 * r**0.75 * i**0.75
    # make nan values zero
    force[np.isnan(force)] = 0
    # return force*10**-6
    return force*10**9

In [564]:
# resolution of the map
res = 200
# random values
size = res * res
size = 10_000
# Seed (if needed)
np.random.seed(42)

# Young's modulus [kPa] - random values following a normal distribution
    #loc: mean/center of distribution
    #scale: std
# Normal distribution: 
'''E = abs(np.random.normal(loc=2.5, scale=1.8, size=size))
# Define a minimum value for a normal distribution, by multiplying lower values by a factor
while np.any(E < 0.2):
    E = np.where(E < 0.2, E*10, E)'''

# Uniform distribution:
# Uniform distribution with random seed
# np.random.seed(42)
# E = abs(np.random.uniform(low=0.2, high=10., size=size))

# Triangular distrbution
E = np.random.triangular(left=0.2, mode=1.8, right=10, size=size)

# Poisson's ratio 
nu = 0.5
# surface energy
    #gamma in (3) is given in J sub-units(???), but gamma <> Ua
# gamma = abs(np.random.normal(loc=0.0001, scale=0.000003, size=size)) f (0,40)
# gamma = abs(np.random.normal(loc=2, scale=0.28, size=size))
gamma = abs(np.random.uniform(low=1, high=3, size=size))
# gamma = 0.1
# radius of the indenter
r = 1980.0 # (nm)

In [565]:
# no contact approach. less points
#linspace(p1, p2, n_pts)
no_contact = np.linspace(-800, 0, 3)

'''DISPLACEMENT VECTORS'''
# xmin, xmax, npts = 0, 250, 50
xmin, xmax, npts = 0, 150, 50

'''Uniformly distributed disp. vectors'''
# indentation depth. more points
contact = np.linspace(xmin, xmax, npts+1)
# approach and withdraw
approach = np.concatenate([no_contact[:-1], contact])
withdraw = np.flip(approach)
ramp = np.concatenate([approach, withdraw])

'''Randomly distributed disp. vectors'''
# Seed (if needed)
np.random.seed(42)

rnd_contact_list = [contact]
for _ in range(size-1):
    aux = np.random.random(npts+1).cumsum()
    aux = (aux-aux.min()) / aux.ptp()     #... .ptp(): peak to peak, i.e., xmax-xmin
    aux = (xmax-xmin)*aux + xmin
    rnd_contact_list.append(aux)
rnd_contact = np.array(rnd_contact_list)
rnd_approach = np.concatenate([np.repeat([no_contact[:-1]], size, axis=0), rnd_contact], axis=1)
rnd_withdraw = np.flip(rnd_approach, axis=1)

# define ramp time
half_cycle = 2 
t_approach = half_cycle*((approach - approach.min(axis=0)) / (approach.max(axis=0) - approach.min(axis=0)))
t_withdraw = half_cycle*((withdraw - withdraw.max(axis=0)) / (withdraw.min(axis=0) - withdraw.max(axis=0)))+max(t_approach)
t = np.concatenate([t_approach, t_withdraw])

In [566]:
# construct dataframe
df = pd.DataFrame()
# 'E' and 'gamma' arrays to list:
df['E'] = E.tolist()
df['gamma'] = gamma.tolist()
# assigns the displacement array for each 'E' (num of E values = len(df) = size)
df['approach'] = [rnd_approach[app] for app in range(len(df))]
df['withdraw'] = [rnd_withdraw[wd] for wd in range(len(df))]
# '..._interp' columns have the sole purpose of allowing the sns errorbar plot 
df['approach_interp'] = [approach for _ in range(len(df))]
df['withdraw_interp'] = [withdraw for _ in range(len(df))]
# applies hertz and jkr models to each row (axis= 0(col) or 1(row))
    # x will take the values of each row 
df['f_hertz'] = df.apply(lambda x: hertz(x.approach, x.E, nu, r), axis=1)
df['f_jkr'] = df.apply(lambda x: jkr(x.withdraw, x.E, nu, x.gamma, r), axis=1)
df['f_hertz_interp'] = df.apply(lambda x: np.interp(x.approach_interp, x.approach, x.f_hertz), axis=1)
df['f_jkr_interp'] = df.apply(lambda x: np.interp(-x.withdraw_interp, -x.withdraw, x.f_jkr), axis=1)

C:\Users\luisr\AppData\Local\Temp\ipykernel_15088\1284831555.py:4: RuntimeWarning: invalid value encountered in power
  force = 4/3 * E / (1 - nu**2) * np.sqrt(r)*i**1.5 * factor
C:\Users\luisr\AppData\Local\Temp\ipykernel_15088\1284831555.py:18: RuntimeWarning: invalid value encountered in power
  force = K * r **0.5 * i**1.5 - Ua * K**0.5 * r**0.75 * i**0.75


In [567]:
#dataframe with contact-only data
#df_jkr: dataframe for jkr data
df_jkr = pd.DataFrame()
df_jkr['withdraw'] = df['withdraw'].copy()
df_jkr['withdraw_contact'] = df_jkr['withdraw'].copy().apply(lambda x: x[x>0])
df_jkr['f_jkr'] = df['f_jkr'].copy()
df_jkr['f_jkr_contact'] = df_jkr['f_jkr'].copy().apply(lambda x: x[:-(len(no_contact))])
df_jkr['E_jkr'] = df['E'].copy()
df_jkr['gamma_jkr'] = df['gamma'].copy()

#check size of disp and force vectors
print(df_jkr['withdraw_contact'][0].shape, df_jkr['f_jkr_contact'][0].shape)
for i in range(size):
    # check if there are negative values in 'f_hertz_contact' 
    if (df_jkr['f_jkr_contact'][i]<0).any():
        print('Negative values in row', i)
        break

(50,) (50,)
Negative values in row 0


In [568]:
def get_mean_std(series: pd.Series):
    return series.mean(), series.std()

def get_max_min(series: pd.Series):
    return series.max(), series.min()

def normalize(x, min, max):
    return (x-min)/(max-min)

def unnormalize(x, min, max):
    return min + x*(max-min)

get_mean_std(df_jkr['E_jkr']), get_mean_std(df_jkr['gamma_jkr']), get_max_min(df_jkr['E_jkr']), get_max_min(df_jkr['gamma_jkr'])

e_max, e_min = get_max_min(df_jkr['E_jkr'])
gamma_max, gamma_min = get_max_min(df_jkr['gamma_jkr'])

In [569]:
test_ratio_jkr = 0.15
valid_ratio_jkr = 0.15
rnd_state_jkr = 42
# SEE INTERSTRAT.ML_STRATIFIERS

target_cols = ['E_jkr_cat', 'gamma_jkr_cat']
nbins = 20

print('Split 1')
while nbins >= 1:
    print(nbins)
    try:
        df_jkr['E_jkr_cat'] = pd.cut(df_jkr['E_jkr'], bins=nbins)
        df_jkr['gamma_jkr_cat'] = pd.cut(df_jkr['gamma_jkr'], bins=nbins)
        train_df_jkr, test_df_jkr = train_test_split(df_jkr, test_size=test_ratio_jkr, 
                                             stratify=df_jkr[target_cols], random_state=rnd_state_jkr)
        break
    except:
        nbins += -1
print('Split 2')
nbins = 20
while nbins >= 1:
    print(nbins)
    try:
        train_df_jkr, valid_df_jkr = train_test_split(train_df_jkr, test_size=valid_ratio_jkr/(1-test_ratio_jkr), 
                                             stratify=train_df_jkr[target_cols], random_state=rnd_state_jkr)
        break
    except:
        nbins += -1

'''while nbins >= 2:
    print(nbins)
    try:
        df_jkr_norm['E_jkr_cat'] = pd.cut(df_jkr_norm['E_jkr_norm'], bins=nbins)
        df_jkr_norm['gamma_jkr_cat'] = pd.cut(df_jkr_norm['gamma_jkr_norm'], bins=nbins)
        train_df_jkr_norm, test_df_jkr = train_test_split(df_jkr_norm, test_size=test_ratio, 
                                             stratify=df_jkr_norm[target_cols], random_state=rnd_state_jkr)
        break
    except:
        nbins += -1

while nbins >= 2:
    print(nbins)
    try:
        train_df_jkr, valid_df_jkr = train_test_split(train_df_jkr_norm, test_size=valid_ratio_jkr/(1-test_ratio_jkr), 
                                             stratify=train_df_jkr_norm[target_cols], random_state=rnd_state_jkr)
        break
    except:
        nbins += -1'''


Split 1
20
19
18
17
16
15
14
13
12
Split 2
20


"while nbins >= 2:\n    print(nbins)\n    try:\n        df_jkr_norm['E_jkr_cat'] = pd.cut(df_jkr_norm['E_jkr_norm'], bins=nbins)\n        df_jkr_norm['gamma_jkr_cat'] = pd.cut(df_jkr_norm['gamma_jkr_norm'], bins=nbins)\n        train_df_jkr_norm, test_df_jkr = train_test_split(df_jkr_norm, test_size=test_ratio, \n                                             stratify=df_jkr_norm[target_cols], random_state=rnd_state_jkr)\n        break\n    except:\n        nbins += -1\n\nwhile nbins >= 2:\n    print(nbins)\n    try:\n        train_df_jkr, valid_df_jkr = train_test_split(train_df_jkr_norm, test_size=valid_ratio_jkr/(1-test_ratio_jkr), \n                                             stratify=train_df_jkr_norm[target_cols], random_state=rnd_state_jkr)\n        break\n    except:\n        nbins += -1"

In [570]:
jkr_df_list = [train_df_jkr, test_df_jkr, valid_df_jkr]
ft_cols = ['withdraw_contact', 'f_jkr_contact']
# lb_cols = ['E_jkr_norm', 'gamma_jkr_norm']
lb_cols = ['E_jkr', 'gamma_jkr']
# lb_cols = ['gamma_jkr', 'E_jkr']
dataset_jkr_list = []

for _, df in enumerate(jkr_df_list):
    aux_arr_ft = np.array(df[ft_cols])
    dataset_jkr_list.append(aux_arr_ft)
    aux_arr_lb = np.array(df[lb_cols])
    dataset_jkr_list.append(aux_arr_lb)

x_train_jkr, y_train_jkr, x_test_jkr, y_test_jkr, x_valid_jkr, y_valid_jkr = dataset_jkr_list

x_train_jkr.shape, y_train_jkr.shape, x_test_jkr.shape, y_test_jkr.shape, x_valid_jkr.shape, y_valid_jkr.shape

((7000, 2), (7000, 2), (1500, 2), (1500, 2), (1500, 2), (1500, 2))

In [571]:
# Training and test data from np arrays to torch tensor with desired shape
def tensor_input_shape(nparray):
    '''
    Input: nparray - numpy array with two dimensions (n_samples, n_features)
    Output: torch_tensor - pytorch tensor with 3 dimensions (n_samples, n_pts, n_features) 
    '''
    n_samples = len(nparray)
    n_pts = len(nparray[0,0])
    torch_tensor = torch.zeros(size=(n_samples, n_pts, 2))
    for i in range(n_samples):
        aux_nparray = np.hstack((np.array(nparray[i,0]).reshape((n_pts,1)), np.array(nparray[i,1]).reshape((n_pts,1))))
        aux_ttensor = torch.from_numpy(aux_nparray).type(torch.float)
        torch_tensor[i,:,:] = aux_ttensor
    return torch_tensor

print(x_train_jkr[1,1].shape)

x_train_t_jkr = tensor_input_shape(x_train_jkr)
x_valid_t_jkr = tensor_input_shape(x_valid_jkr)
x_test_t_jkr = tensor_input_shape(x_test_jkr)
y_train_t_jkr = torch.from_numpy(y_train_jkr).type(torch.float)
y_valid_t_jkr = torch.from_numpy(y_valid_jkr).type(torch.float)
y_test_t_jkr = torch.from_numpy(y_test_jkr).type(torch.float)
#x_train_t2 = torch.from_numpy(x_train).type(torch.float)

x_train_t_jkr.shape, y_train_t_jkr.shape, y_valid_t_jkr.shape, y_test_t_jkr.shape

(50,)


(torch.Size([7000, 50, 2]),
 torch.Size([7000, 2]),
 torch.Size([1500, 2]),
 torch.Size([1500, 2]))

In [572]:
def create_model_dir(timestamp, contact_model: str):

  ''' Second input must be 'hertz' or 'jkr' '''
  
  allowed_models = ['hertz', 'jkr']
  if contact_model not in allowed_models:
    raise ValueError("Input value must be one of %s" % allowed_models)
  model_path = 'model_{}'.format(timestamp)
  parent_dir = 'c:\\Users\\luisr\\OneDrive\\Ambiente de Trabalho\\Tese'
  if contact_model == 'hertz':
    dir = 'Hertz_models'
  elif contact_model == 'jkr':
    dir = 'JKR_models'
  path = os.path.join(parent_dir, dir, model_path)
  # path = os.path.join(initial_wd, dir, model_path)
  os.mkdir(path)
  os.chdir(path)

def error_fn(predict_tensor, label_tensor):
  '''
  INPUTS: * two tensors - true labels and predicts
  OUTPUTS: * scalar - mean relative error (in %) between both tensors
           * list - relative error (%) for each prediction
  '''
  error = abs((label_tensor-predict_tensor)/label_tensor*100).squeeze(dim=1).mean().item()
  error_list = list(abs((label_tensor-predict_tensor)/label_tensor*100).squeeze(dim=1).detach().numpy())
  return error, error_list

In [573]:
class JKR_Dataset():
  
  def __init__(self,features,labels):
    self.features = features
    self.labels = labels
 
  def __len__(self):
    return len(self.labels)
   
  def __getitem__(self,idx):
    return self.features[idx],self.labels[idx]
  
train_data_jkr = JKR_Dataset(x_train_t_jkr, y_train_t_jkr)
test_data_jkr = JKR_Dataset(x_test_t_jkr, y_test_t_jkr)
valid_data_jkr = JKR_Dataset(x_valid_t_jkr, y_valid_t_jkr)

In [574]:
################ After changing one of the hyperparameters: ########################
### Re-run the cells where the model class and the model_params dict are defined ###
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"

# HYPERPARAMETERS
LEARNING_RATE_JKR = 0.0005
EPOCHS_JKR = 50
BATCH_SIZE_JKR = 128

# Size of each layer
HIDDEN_UNITS_1_JKR = 512
HIDDEN_UNITS_2_JKR = 256
HIDDEN_UNITS_3_JKR = 64

ARCHITECTURE_JKR = 2

DEVICE

'cuda'

In [575]:
train_loader_jkr=DataLoader(train_data_jkr,batch_size=BATCH_SIZE_JKR,shuffle=True)
test_loader_jkr=DataLoader(test_data_jkr,batch_size=int(test_ratio_jkr*size+1),shuffle=False)
valid_loader_jkr=DataLoader(valid_data_jkr, batch_size=int(valid_ratio_jkr*size+1), shuffle=False)

In [576]:
class MAPELoss(nn.Module):
    def __init__(self):
        super(MAPELoss, self).__init__()
        self.mape = nn.L1Loss(reduction='none')
    
    def forward(self, input, target):
        mape = self.mape(input, target)
        norm_factor = torch.abs(target)
        mape = (mape / norm_factor)
        return mape

In [577]:
class Regression_JKR(nn.Module):
    def __init__(self, input_shape, HIDDEN_UNITS_1_JKR, HIDDEN_UNITS_2_JKR, HIDDEN_UNITS_3_JKR):
        super(Regression_JKR, self).__init__()
        input_size = input_shape[0] * input_shape[1]
        self.layers = nn.Sequential(nn.Flatten(),
                                    nn.Linear(input_size, HIDDEN_UNITS_1_JKR),
                                    nn.ReLU(),
                                    nn.Linear(HIDDEN_UNITS_1_JKR,HIDDEN_UNITS_2_JKR),
                                    nn.ReLU(),
                                    nn.Linear(HIDDEN_UNITS_2_JKR,HIDDEN_UNITS_3_JKR),
                                    nn.ReLU(),
                                    nn.Linear(HIDDEN_UNITS_3_JKR, 2))
    def forward(self, x):
        out = self.layers(x)
        return out
# Define input shape
input_shape_jkr = x_train_t_jkr.shape[1:]

# Instantiate the model
torch.manual_seed(42)
model_jkr = Regression_JKR(input_shape_jkr, HIDDEN_UNITS_1_JKR, HIDDEN_UNITS_2_JKR, HIDDEN_UNITS_3_JKR)


# Experimentar Huber Loss, log-cosh e quantile loss !!!!!!!!!!
# Define the loss function and optimizer
loss_fn_jkr = nn.MSELoss(reduction='none')
# loss_fn_jkr = nn.MSELoss()
# loss_fn_jkr = nn.L1Loss(reduction='none')

optimizer_jkr = torch.optim.Adam(model_jkr.parameters(),
                                lr=LEARNING_RATE_JKR)

In [578]:
'''class Regression_JKR_norm(nn.Module):
    def __init__(self, input_shape, HIDDEN_UNITS_1_JKR, HIDDEN_UNITS_2_JKR, HIDDEN_UNITS_3_JKR):
        super(Regression_JKR_norm, self).__init__()
        input_size = input_shape[0] * input_shape[1]
        self.layers = nn.Sequential(nn.Flatten(),
                                    nn.Linear(input_size, HIDDEN_UNITS_1_JKR),
                                    nn.ReLU(),
                                    nn.Linear(HIDDEN_UNITS_1_JKR,HIDDEN_UNITS_2_JKR),
                                    nn.ReLU(),
                                    nn.Linear(HIDDEN_UNITS_2_JKR,HIDDEN_UNITS_3_JKR),
                                    nn.ReLU(),
                                    nn.Linear(HIDDEN_UNITS_3_JKR, 2),
                                    nn.BatchNorm1d(2))
    def forward(self, x):
        out = self.layers(x)
        return out
# Define input shape
input_shape_jkr = x_train_t_jkr.shape[1:]

# Instantiate the model
torch.manual_seed(42)
model_jkr_norm = Regression_JKR_norm(input_shape_jkr, HIDDEN_UNITS_1_JKR, HIDDEN_UNITS_2_JKR, HIDDEN_UNITS_3_JKR)

# Define the loss function and optimizer
# loss_fn_jkr_norm = nn.MSELoss(reduction='none')
# loss_fn_jkr_norm = nn.MSELoss()
loss_fn_jkr_norm = nn.MSELoss(reduction='none')

optimizer_jkr_norm = torch.optim.SGD(model_jkr_norm.parameters(),
                                lr=LEARNING_RATE_JKR)'''

"class Regression_JKR_norm(nn.Module):\n    def __init__(self, input_shape, HIDDEN_UNITS_1_JKR, HIDDEN_UNITS_2_JKR, HIDDEN_UNITS_3_JKR):\n        super(Regression_JKR_norm, self).__init__()\n        input_size = input_shape[0] * input_shape[1]\n        self.layers = nn.Sequential(nn.Flatten(),\n                                    nn.Linear(input_size, HIDDEN_UNITS_1_JKR),\n                                    nn.ReLU(),\n                                    nn.Linear(HIDDEN_UNITS_1_JKR,HIDDEN_UNITS_2_JKR),\n                                    nn.ReLU(),\n                                    nn.Linear(HIDDEN_UNITS_2_JKR,HIDDEN_UNITS_3_JKR),\n                                    nn.ReLU(),\n                                    nn.Linear(HIDDEN_UNITS_3_JKR, 2),\n                                    nn.BatchNorm1d(2))\n    def forward(self, x):\n        out = self.layers(x)\n        return out\n# Define input shape\ninput_shape_jkr = x_train_t_jkr.shape[1:]\n\n# Instantiate the model\ntorch.manu

In [579]:
def train_one_epoch_jkr(epoch_index, train_loader): # (epoch_index, tb_writer)
    # running_loss = 0.
    # last_loss = 0.
    loss_list = []
    error_E_list = []
    error_gamma_list = []
    for i, data in enumerate(train_loader):
        # Every data instance is an input + label pair
        inputs, labels = data
        optimizer_jkr.zero_grad()
        predicts = model_jkr(inputs)
        # print(predicts)
        # Compute the loss and its gradients
        loss = loss_fn_jkr(predicts, labels).mean(dim=0)
        '''max, _ = loss_fn_jkr(predicts, labels).max(0)
        loss = (loss_fn_jkr(predicts, labels)/max).mean()'''
        '''loss1, loss2 = loss_fn_jkr(predicts[:,0], labels[:,0]), loss_fn_jkr(predicts[:,1], labels[:,1])
        loss = loss1 * loss2'''
        error_E, _ = error_fn(predicts[:,0].unsqueeze(dim=1), labels[:,0].unsqueeze(dim=1))
        error_gamma, _ = error_fn(predicts[:,1].unsqueeze(dim=1), labels[:,1].unsqueeze(dim=1))
        loss.backward(gradient=torch.Tensor([1., 1.]))
        # Adjust learning weights
        optimizer_jkr.step()
        # Gather data and report
        loss_list.append(loss.detach().numpy())
        error_E_list.append(error_E)
        error_gamma_list.append(error_gamma)
        # running_loss += loss.item()  # .item() converts tensor to number
        # print(i, loss.item())
    return loss_list, error_E_list, error_gamma_list

In [580]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    # n_layers = trial.suggest_int("n_layers", 3, 4)
    n_layers = 3
    layers = []
    neurons = []
    in_features = input_shape_jkr[0] * input_shape_jkr[1]
    for i in range(n_layers):
        if i == 0:
            layers.append(nn.Flatten())
            out_features = 128
            # out_features = trial.suggest_categorical("n_units_l{}".format(i), [64, 128, 256])
        elif i == 1 or i == 2:
            out_features = 128
        elif i == 2:
            out_features = 64
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        neurons.append(out_features)
        # p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        # layers.append(nn.Dropout(p))
        in_features = out_features
    layers.append(nn.Linear(in_features, 2))
    # layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers), neurons

In [581]:

n_layers_list = []
lr_list = []
nl1_list, nl2_list, nl3_list = [], [], []
epoch_list = []
def objective(trial):

    # model_Hertz = Regression_Hertz(input_shape, HIDDEN_UNITS_1, HIDDEN_UNITS_2).to(DEVICE)
    model_JKR = define_model(trial)[0].to(DEVICE)
    
    learning_rate = trial.suggest_float("learning_rate", 2e-4, 5e-3, log=True)
    # n_epochs = trial.suggest_int('n_epochs', 90, 140, step=10)
    n_epochs = 120
    # optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD"])
    # batch_size = trial.suggest_categorical("batch_size", [16, 32])
    batch_size = 16
    # n_layers = trial.suggest_int("n_layers", 3, 4)
    # n_nodes = trial.suggest_int("n_nodes", 4, 64, log=True)
    train_loader_jkr=DataLoader(train_data_jkr,batch_size=batch_size,shuffle=True)
    ## loss_fn = nn.HuberLoss()
    loss_fn = nn.MSELoss(reduction='none')
    optimizer = torch.optim.Adam(model_JKR.parameters(), lr=learning_rate)
    # loss_fn = MAPELoss()
    # optimizer = getattr(optim, optimizer_name)(model_JKR.parameters(), lr=learning_rate)
    epoch_list.append(n_epochs)
    lr_list.append(learning_rate)
    nl1_list.append(define_model(trial)[1][0])
    nl2_list.append(define_model(trial)[1][1])
    nl3_list.append(define_model(trial)[1][2])
    for epoch in range(n_epochs):
        model_JKR.train(True)
        loss_list = []
        error_list = []
        for i, data in enumerate(train_loader_jkr):
            # Every data instance is an input + label pair
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model_JKR(inputs.to(DEVICE))
            # print(f'Outputs: {outputs.mean()}')
            # Compute the loss and its gradients
            ## loss = loss_fn(outputs, labels.to(DEVICE)).mean()
            loss = loss_fn(outputs, labels.to(DEVICE)).mean(dim=0)
            # error, _ = error_fn(outputs, labels)
            loss.backward(gradient=torch.Tensor([1., 1.]).to(DEVICE))
            ##loss.backward()
            ## loss_list.append(loss.item())
            # Adjust learning weights
            optimizer.step()
        # Evaluation
        model_JKR.eval()
        running_vloss = 0.0
        # running_verror_E, running_verror_gamma = 0.0, 0.0
        verror_list, fts_list, labels_list, predicts_list = [], [], [], []
        with torch.no_grad():
            for i, vdata in enumerate(valid_loader_jkr):
                vinputs, vlabels = vdata
                voutputs = model_JKR(vinputs.to(DEVICE))
                predicts_list.append(voutputs)
                vloss = loss_fn(voutputs, vlabels.to(DEVICE)).mean()
                # print(f'Loss: {vloss}')
                # verror, verror_aux_list = error_fn(voutputs, vlabels)
                # error_E, _ = error_fn(voutputs[:,0].unsqueeze(dim=1).to(DEVICE), vlabels[:,0].unsqueeze(dim=1).to(DEVICE))
                # error_gamma, _ = error_fn(voutputs[:,1].unsqueeze(dim=1).to(DEVICE), vlabels[:,1].unsqueeze(dim=1).to(DEVICE))
                running_vloss += vloss.item()
                # running_verror_gamma += error_gamma
                # running_verror_E += error_E
                # verror_list += verror_aux_list
            loss = running_vloss / (i + 1)
            # verror_E = running_verror_E / (i + 1)
            # verror_gamma = running_verror_gamma / (i + 1)
            # avg_verror = ((verror_E + verror_gamma) / 2)
            # print(loss.item())
            # avg_verror = running_verror / (i + 1)
            # print(avg_verror)
        # trial.report(loss.item(), epoch)
        trial.report(loss, epoch)
        # trial.report(avg_verror, epoch)
        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    # return loss
    return loss

if __name__ == "__main__":
    study_name = "JKR_Huber_LeakyReLU"
    study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=2), 
                                study_name=study_name)
    study.optimize(objective, n_trials=100)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    losses = [trial.value for trial in study.trials]
    losses_array = np.array(losses)
    lr_array = np.array(lr_list)
    # nl1_array = np.array(nl1_list)
    # nl2_array = np.array(nl2_list)
    # nl3_array = np.array(nl3_list)
    # epoch_array = np.array(epoch_list)
    
    np.save("losses.npy", losses_array)
    np.save("lr.npy", lr_array)
    # np.save("nl1.npy", nl1_array)
    # np.save("nl2.npy", nl2_array)
    # np.save("nl3.npy", nl3_array)
    # np.save("epoch.npy", epoch_array)

[I 2023-06-15 15:25:15,169] A new study created in memory with name: JKR_Huber_LeakyReLU
[I 2023-06-15 15:27:36,170] Trial 0 finished with value: 0.01817111112177372 and parameters: {'learning_rate': 0.0007522660787915097}. Best is trial 0 with value: 0.01817111112177372.
[I 2023-06-15 15:29:49,782] Trial 1 finished with value: 0.0023690471425652504 and parameters: {'learning_rate': 0.0008002728719926263}. Best is trial 1 with value: 0.0023690471425652504.
[I 2023-06-15 15:32:05,552] Trial 2 finished with value: 0.005737136583775282 and parameters: {'learning_rate': 0.0022946667087656155}. Best is trial 1 with value: 0.0023690471425652504.
[I 2023-06-15 15:34:24,884] Trial 3 finished with value: 0.007150169461965561 and parameters: {'learning_rate': 0.0004970912345515033}. Best is trial 1 with value: 0.0023690471425652504.
[I 2023-06-15 15:36:48,595] Trial 4 finished with value: 0.02251722663640976 and parameters: {'learning_rate': 0.0002688385028753457}. Best is trial 1 with value: 0.

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  94
  Number of complete trials:  6
Best trial:
  Value:  0.0023690471425652504
  Params: 
    learning_rate: 0.0008002728719926263


In [582]:
a = torch.tensor([1,2,3], dtype=torch.float32, requires_grad=True)
a.detach()

tensor([1., 2., 3.])